In [1]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# housing data import and add bias ahead of matrix
housing = fetch_california_housing()
m, n = housing.data.shape

In [2]:
n_epochs = 10000
learning_rate = 0.01

In [3]:
scaled_housing_data = housing.data / housing.data.max(axis=0)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]
#scaled_housing_data_plus_bias

In [4]:
%%time
tf.compat.v1.disable_v2_behavior()
## tensorflow model
X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")

## random support of theta
theta = tf.Variable(tf.random.normal([n+1, 1], -1.0, 1.0), name="theta")

## random support of theta
y_pred = tf.matmul(X, theta, name="preditions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.compat.v1.assign(theta, theta - learning_rate * gradients)

init = tf.compat.v1.global_variables_initializer()

with tf.compat.v1.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 1000 == 0:
            print("Epoch {}, MSE = {}".format(epoch, mse.eval()))
        sess.run(training_op)
        
    best_beta = theta.eval()
    print("best beta : {}".format(best_beta))

Instructions for updating:
non-resource variables are not supported in the long term
Epoch 0, MSE = 31.44782066345215
Epoch 1000, MSE = 0.9920492768287659
Epoch 2000, MSE = 0.8349885940551758
Epoch 3000, MSE = 0.7504395246505737
Epoch 4000, MSE = 0.7044603824615479
Epoch 5000, MSE = 0.6792659163475037
Epoch 6000, MSE = 0.6653107404708862
Epoch 7000, MSE = 0.657444179058075
Epoch 8000, MSE = 0.6528835892677307
Epoch 9000, MSE = 0.6501245498657227
best beta : [[ 0.30830124]
 [ 6.2135396 ]
 [ 0.8407393 ]
 [-0.8040251 ]
 [ 0.16767122]
 [-0.92598575]
 [-0.3661116 ]
 [-0.74829674]
 [ 0.3750848 ]]
CPU times: user 6.87 s, sys: 1.45 s, total: 8.32 s
Wall time: 5.19 s
